In [1]:
import pandas as pd
import re
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
data = pd.read_csv('train.csv')
ps = PorterStemmer()

In [2]:
def all_the_cooking(line):
    reviews = re.sub('[^a-zA-Z]' , ' ', line)
    reviews = reviews.lower()
    reviews = reviews.split()
    reviews = [ps.stem(word) for word in reviews if not word in set(stopwords.words('english'))]
    reviews = ' '.join(reviews)
    return(reviews)

In [3]:
for i,line in enumerate(data.tweet):
    data.tweet[i] = all_the_cooking(line)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [4]:
vect = TfidfVectorizer()
tfid_data_x = vect.fit_transform(data['tweet'])
train_y  = data['label']

In [6]:
from imblearn.over_sampling import SMOTENC

In [7]:
smnc = SMOTENC(categorical_features =[0,2] , random_state = 0)
smnc_xtrain_tfid, smnc_trainy = smnc.fit_sample(tfid_data_x, train_y)

In [9]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(smnc_xtrain_tfid, smnc_trainy)
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X=train_x , y = train_y)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [10]:
expected = test_y
predicted=model.predict(test_x)

In [11]:
from sklearn import metrics

print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))


              precision    recall  f1-score   support

           0       1.00      0.94      0.97      7453
           1       0.94      1.00      0.97      7407

    accuracy                           0.97     14860
   macro avg       0.97      0.97      0.97     14860
weighted avg       0.97      0.97      0.97     14860

[[7021  432]
 [  27 7380]]


In [12]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_y, predicted)

In [13]:
accuracy

0.9691117092866757

In [14]:
metrics.f1_score(test_y,predicted)

0.9698403311649912